In [35]:
# update signals.json for web screener
# Daily Looping
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
import datetime
import pandas_ta as ta
import tqdm
import subprocess
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def get_tickers(file="SP500"):
    dict = pd.read_csv(filemap[file], header=None)
    dict = dict.iloc[:, 0]
    return dict

def WPRbt(data):
    period = 12
    max = data.Close.rolling(period).max()
    min = data.Close.rolling(period).min()
    wpr = (max - data.Close)/(max - min) * (-100) 
    return wpr

def check_WPR_2MA_without_DB(ticker, test=False, period_long_ma=28, period_short_ma=14, upb=-20, lob=-80):
    a = yf.download(ticker, start=datetime.datetime.now() - datetime.timedelta(days=50), interval="1d", progress=False)

    # a["WPR"] = WPRbt(a)
    a["MAX"] = a.High.rolling(12).max()
    a["MIN"] = a.Low.rolling(12).min()
    a["WPRs"] = (a.MAX - a.Close)/(a.MAX - a.MIN) * (-100)
    a["long_MA"] = a.Close.ewm(span=period_long_ma, adjust=False, min_periods=period_long_ma).mean()
    a['short_MA'] = a.Close.ewm(span=period_short_ma, adjust=False, min_periods=period_short_ma).mean()
    long0 = a['long_MA'][-1]
    long1 = a['long_MA'][-2]
    long2 = a['long_MA'][-3]
    long3 = a['long_MA'][-4]
    # =======================
    short0 = a['short_MA'][-1]
    short1 = a['short_MA'][-2]
    short2 = a['short_MA'][-3]
    short3 = a['short_MA'][-4]
    # =======================
    wpr0 = a['WPRs'][-1]
    wpr1 = a['WPRs'][-2]
    wpr2 = a['WPRs'][-3]
    wpr3 = a['WPRs'][-4]
    check_ma_percent = 0.0000# for PAYX at 2024-02-08, 0.005 return 4(meaning doesn't work ) while 0.006 works
    bear_market = (long0 - short0 > long0*check_ma_percent) and (long1 - short1 > long1*check_ma_percent) and (long2 - short2 > long2*check_ma_percent)
    bull_market = (long0 - short0 < long0*-check_ma_percent) and (long1 - short1 < long1*-check_ma_percent) and (long2 - short2 < long2*-check_ma_percent)
    if test:
        print(a[['Close', "short_MA", "long_MA", "WPRs", "MAX"]])
        print(wpr0, wpr1, wpr2, wpr3)
        print("quick: ", [short3, short2, short1, short0])
        print("slow: ", [long3, long2, long1, long0])
    # current signal
    if((wpr0 < upb) and (wpr1 > upb) and bear_market):
        # print(ticker, "cross above"):
            # sell signal
        return 1
    elif ((wpr0 > lob) and (wpr1 < lob)) and bull_market:
        # buy signal
        return 2
    # check for yesterday or the day before yesterday buy/sell signal
    elif((wpr1 < upb) and (wpr2 > upb) and bear_market):
        return 1.5
    elif((wpr1 > lob) and (wpr2 < lob) and bull_market):
        return 2.5
    elif((wpr2 > lob) and (wpr3 < lob) and bull_market) or ((wpr2 > upb) and (wpr3 < upb) and bear_market):
        return 3

    else:
        return 0

def mains(loops = ["NYSE", "NASDAQ", "SP500"], updateDatabase = False):
    all_signals_sell = {}
    all_signals_buy = {}
    for k in loops:
        print("*"*20, k, "*"*20)

        with open(filemap[k], 'r') as f:
            python_dict = json.load(f)
        a = list(python_dict.keys())
        if k in ['portfolio', "SP500", "Yuanta"]:
            tickers = a[:]
        elif k == "NYSE":
            tickers = a[1:1352]
        elif k == "NASDAQ":
            tickers = a[1:1255]
        with tqdm.tqdm(total=len(tickers)) as pbar:
            res = {}
            for ticker in tickers:
                res[ticker] = check_WPR_2MA_without_DB(ticker)
                pbar.update(1)  # Update the progress bar

        # s = {{i:res[i]} for i in res.keys() if res[i] in [1, 2]}
        for key, value in res.items():

            if value in [1, 2]:
                command = "zacks-api " + key
                result = subprocess.check_output(command, shell=True)
                dict = eval(result.decode(("utf-8")))
                rank = dict["zacksRank"]
                if value == 1:
                    all_signals_sell[key] = rank
                elif value == 2:
                    all_signals_buy[key] = rank
        if sum(list(res.values())) == 0:
            print("there's nothing with signal")

    time = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    with open("../Database/signals.json", "r") as f:
        python_dict_signal = json.load(f)
    python_dict_signal[time] = {"buy": all_signals_buy, "sell": all_signals_sell} 
    with open("../Database/signals.json", "w") as f:
        json.dump(python_dict_signal, f , indent=4)
    print({"buy": all_signals_buy, "sell": all_signals_sell})
updateDatabase = True

loops = ["Yuanta", "SP500"]
mains(updateDatabase=updateDatabase, loops=loops)


******************** Yuanta ********************


100%|██████████| 18/18 [00:02<00:00,  7.24it/s]


******************** SP500 ********************


100%|██████████| 499/499 [01:14<00:00,  6.66it/s]


{'buy': {'AMZN': '1', 'MA': '2', 'NOW': '3', 'PSX': '3', 'MPC': '3', 'AZO': '3', 'VLO': '3', 'TEL': '2', 'DLR': '4', 'CPRT': '3', 'SYY': '3', 'CLX': '2', 'TXT': '1', 'JBHT': '3'}, 'sell': {'PSA': '3', 'TAP': '1'}}


In [3]:
# update signals.json for web screener
# Daily Looping
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
import datetime
import pandas_ta as ta
import tqdm
import subprocess
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def get_tickers(file="SP500"):
    dict = pd.read_csv(filemap[file], header=None)
    dict = dict.iloc[:, 0]
    return dict

def WPRbt(data):
    period = 12
    max = data.Close.rolling(period).max()
    min = data.Close.rolling(period).min()
    wpr = (max - data.Close)/(max - min) * (-100) 
    return wpr

def check_WPR_2MA_without_DB(ticker, test=False, period_long_ma=28, period_short_ma=14, upb=-20, lob=-80):
    a = yf.download(ticker, start=datetime.datetime.now() - datetime.timedelta(days=50), interval="1d", progress=False)

    # a["WPR"] = WPRbt(a)
    a["MAX"] = a.High.rolling(12).max()
    a["MIN"] = a.Low.rolling(12).min()
    a["WPRs"] = (a.MAX - a.Close)/(a.MAX - a.MIN) * (-100)
    a["long_MA"] = a.Close.ewm(span=period_long_ma, adjust=False, min_periods=period_long_ma).mean()
    a['short_MA'] = a.Close.ewm(span=period_short_ma, adjust=False, min_periods=period_short_ma).mean()
    long0 = a['long_MA'][-1]
    long1 = a['long_MA'][-2]
    long2 = a['long_MA'][-3]
    long3 = a['long_MA'][-4]
    # =======================
    short0 = a['short_MA'][-1]
    short1 = a['short_MA'][-2]
    short2 = a['short_MA'][-3]
    short3 = a['short_MA'][-4]
    # =======================
    wpr0 = a['WPRs'][-1]
    wpr1 = a['WPRs'][-2]
    wpr2 = a['WPRs'][-3]
    wpr3 = a['WPRs'][-4]
    check_ma_percent = 0.0000# for PAYX at 2024-02-08, 0.005 return 4(meaning doesn't work ) while 0.006 works
    bear_market = (long0 - short0 > long0*check_ma_percent) and (long1 - short1 > long1*check_ma_percent) and (long2 - short2 > long2*check_ma_percent)
    bull_market = (long0 - short0 < long0*-check_ma_percent) and (long1 - short1 < long1*-check_ma_percent) and (long2 - short2 < long2*-check_ma_percent)
    if test:
        print(a[['Close', "short_MA", "long_MA", "WPRs", "MAX"]])
        print(wpr0, wpr1, wpr2, wpr3)
        print("quick: ", [short3, short2, short1, short0])
        print("slow: ", [long3, long2, long1, long0])
    # current signal
    if((wpr0 < upb) and (wpr1 > upb) and bear_market):
        # print(ticker, "cross above"):
            # sell signal
        return 1
    elif ((wpr0 > lob) and (wpr1 < lob)) and bull_market:
        # buy signal
        return 2
    # check for yesterday or the day before yesterday buy/sell signal
    elif((wpr1 < upb) and (wpr2 > upb) and bear_market):
        return 1.5
    elif((wpr1 > lob) and (wpr2 < lob) and bull_market):
        return 2.5
    elif((wpr2 > lob) and (wpr3 < lob) and bull_market) or ((wpr2 > upb) and (wpr3 < upb) and bear_market):
        return 3

    else:
        return 0

def mains(loops = ["NYSE", "NASDAQ", "SP500"], updateDatabase = False):
    all_signals_sell = {}
    all_signals_buy = {}
    for k in loops:
        print("*"*20, k, "*"*20)

        with open(filemap[k], 'r') as f:
            python_dict = json.load(f)
        a = list(python_dict.keys())
        if k in ['portfolio', "SP500", "Yuanta"]:
            tickers = a[:]
        elif k == "NYSE":
            tickers = a[1:1352]
        elif k == "NASDAQ":
            tickers = a[1:1255]
        with tqdm.tqdm(total=len(tickers)) as pbar:
            res = {}
            for ticker in tickers:
                res[ticker] = check_WPR_2MA_without_DB(ticker)
                pbar.update(1)  # Update the progress bar

        # s = {{i:res[i]} for i in res.keys() if res[i] in [1, 2]}
        for key, value in res.items():
            try:
                if value in [1, 2]:
                    command = "zacks-api " + key
                    result = subprocess.check_output(command, shell=True)
                    dict = eval(result.decode(("utf-8")))
                    rank = dict["zacksRank"]
                    if value == 1:
                        all_signals_sell[key] = rank
                    elif value == 2:
                        all_signals_buy[key] = rank
                if sum(list(res.values())) == 0:
                    print("there's nothing with signal")
            except:
                print(key, "false")
    time = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    with open("../Database/signals.json", "r") as f:
        python_dict_signal = json.load(f)
    python_dict_signal[time] = {"buy": all_signals_buy, "sell": all_signals_sell} 
    with open("../Database/signals.json", "w") as f:
        json.dump(python_dict_signal, f , indent=4)
    print({"buy": all_signals_buy, "sell": all_signals_sell})
updateDatabase = True

loops = ["SP500", "NASDAQ", "NYSE"]
mains(updateDatabase=updateDatabase, loops=loops)


******************** SP500 ********************


  0%|          | 0/499 [00:00<?, ?it/s]

100%|██████████| 499/499 [01:20<00:00,  6.19it/s]


******************** NASDAQ ********************


100%|██████████| 1254/1254 [02:43<00:00,  7.66it/s]


******************** NYSE ********************


100%|██████████| 1351/1351 [03:20<00:00,  6.73it/s]


{'buy': {'RTX': '3', 'GPN': '3', 'ETSY': '3', 'DKNG': '3', 'KPTI': '4', 'MLCO': '3', 'RPD': '1', 'KURA': '3', 'TSVT': '3', 'MREO': '3', 'CRNX': '3', 'ORIC': '3', 'PI': '2', 'OSW': '2', 'ICCM': '3', 'ITI': '4', 'DOMO': '3', 'VRA': '3', 'OFIX': '3', 'PRGS': '3', 'AKYA': '2', 'BJRI': '3', 'CNHI': '3', 'S': '3', 'SMR': '3', 'CVI': '4', 'EB': '2', 'FOUR': '4', 'OSK': '2', 'IGT': '4', 'SEM': '4', 'RDW': '3', 'DCI': '2', 'CNA': '3'}, 'sell': {'TXN': '5', 'UPS': '3', 'SCHW': '4', 'SLB': '3', 'EOG': '4', 'MRNA': '3', 'RCL': '1', 'WY': '3', 'INCY': '3', 'UDR': '4', 'MOS': '3', 'TSP': '3', 'DADA': '3', 'ONB': '3', 'BMRN': '3', 'FIBK': '3', 'FLWS': '3', 'METC': '3', 'ZNTL': '2', 'ALEC': '3', 'PMVP': '3', 'RNAZ': '3', 'TRIN': '2', 'KC': '3', 'UPBD': '4', 'OTRK': '3', 'AA': '3', 'CRK': '4', 'W': '2', 'DO': '4', 'PK': '3', 'MUR': '5', 'TALO': '3', 'FNB': '2', 'CADE': '3', 'COLD': '2', 'NFG': '5', 'BKU': '3', 'CRGY': '3', 'VET': '4', 'KRC': '4', 'DRH': '3', 'FTS': '3', 'APLE': '3', 'FN': '2', 'XHR': '

In [ ]:
# update Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def daily_add_datetime_minus_1(loops=list(filemap.keys())):
    for kys in loops:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        i = 0
        for ticker in a:
            df = pd.read_csv(f"../Database/Tickers/{ticker}.csv", index_col="Date")
            start = datetime.datetime.now()-datetime.timedelta(days=1)
            ftime = start.strftime("%Y-%m-%d")
            try:
                b = df.loc[ftime]
            except KeyError:
                a = yf.download("A", start=start)
                a.index = [ftime]
                df = pd.concat([df, a])
                df.index.name = "Date"
                df.to_csv("../Database/Tickers/A.csv", index="Date")
                if i == 0:
                    print(f"added on {kys}")
                    i += 1
daily_add_datetime_minus_1()                

In [ ]:
# initialization of Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def database_initialization():
    for kys in ["NYSE", "NASDAQ", "SP500"]:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        a = list(python_dict.keys())
        for ticker in a:
            print(ticker)
            df = yf.download(ticker, progress=False)
            df.to_csv(f"../Database/Tickers/{ticker}.csv")
        
database_initialization()                
